# Face Emotion Recognition

https://huggingface.co/datasets/tukey/human_face_emotions_roboflow/viewer/default/train?p=1&views%5B%5D=train

In [23]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Import Data

In [24]:
import pandas as pd
import io
from PIL import Image

df = pd.read_parquet("hf://datasets/tukey/human_face_emotions_roboflow/data/train-00000-of-00001.parquet")

# Data Overview & Cleaning

In [25]:
# Standardize column names (strip whitespace, lower-case, replace spaces with underscores)
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

# Check for missing values
print("Missing values per column:")
print(df.isna().sum())

# No missing values or duplicates, so we can proceed with the data as is

# Print out summary information
print("\nDataframe Info:")
print(df.info())

# Print the first few rows to inspect the data
print("\nFirst 5 rows of the dataset:")
print(df.head())

Missing values per column:
image    0
qa       0
dtype: int64

Dataframe Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9400 entries, 0 to 9399
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   image   9400 non-null   object
 1   qa      9400 non-null   object
dtypes: object(2)
memory usage: 147.0+ KB
None

First 5 rows of the dataset:
                                               image  \
0  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...   
1  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...   
2  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...   
3  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...   
4  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...   

                                                  qa  
0  [{'question': 'How does the person feel in the...  
1  [{'question': 'How does the person feel in the...  
2  [{'question': 'How does the person feel in the...  
3  [{'question': 'How does the person

In [26]:
# Check for missing values in each column
print("\nMissing values per column:")
print(df.isnull().sum())


Missing values per column:
image    0
qa       0
dtype: int64


In [27]:
import json
import numpy as np

def extract_emotion(qa_entry):
    try:
        # If the qa_entry is a string, strip it and parse as JSON.
        if isinstance(qa_entry, str):
            qa_entry = qa_entry.strip()
            qa_data = json.loads(qa_entry)
        else:
            qa_data = qa_entry

        # If the data is a numpy array, convert it to a list.
        if isinstance(qa_data, np.ndarray):
            qa_data = qa_data.tolist()

        # Now you can check if it's a list or tuple using this condition.
        if isinstance(qa_data, (list, tuple)) and len(qa_data) > 0:
            return qa_data[0].get("answer")
        else:
            print("Unexpected qa_data structure:", qa_data, "with type", type(qa_data))
    except Exception as e:
        print("Error parsing qa entry:", qa_entry, "\nError:", e)
    return None

In [28]:
# Assuming df is your DataFrame that includes the 'qa' column
df["emotion"] = df["qa"].apply(extract_emotion)

# Verify the new column
print(df[["qa", "emotion"]].head())

                                                  qa  emotion
0  [{'question': 'How does the person feel in the...      sad
1  [{'question': 'How does the person feel in the...    anger
2  [{'question': 'How does the person feel in the...  neutral
3  [{'question': 'How does the person feel in the...     fear
4  [{'question': 'How does the person feel in the...  content


In [29]:
# Check unique values and distribution of facial emotion labels
if 'emotion' in df.columns:
    print("\nUnique emotion labels:")
    print(df['emotion'].unique())

    print("\nDistribution of emotion labels:")
    print(df['emotion'].value_counts())


Unique emotion labels:
['sad' 'anger' 'neutral' 'fear' 'content' 'happy' 'disgust' 'surprise']

Distribution of emotion labels:
emotion
surprise    1238
neutral     1225
sad         1184
fear        1181
anger       1175
disgust     1165
content     1144
happy       1088
Name: count, dtype: int64


In [30]:
import matplotlib.pyplot as plt

# Example: Plot a histogram for a numeric column, adjust 'score' to the relevant column name
if 'score' in df.columns:
    plt.hist(df['score'].dropna(), bins=30, edgecolor='k')
    plt.xlabel("Score")
    plt.ylabel("Frequency")
    plt.title("Histogram of Scores")
    plt.show()

In [31]:
# Optionally, save the cleaned dataframe to disk as a new parquet file or CSV
df.to_parquet("cleaned_human_face_emotions.parquet")
# Alternatively, you can save as CSV:
# df.to_csv("cleaned_human_face_emotions.csv", index=False)

# drop qa column
df.drop(columns=["qa"], inplace=True)

print(df.head())

                                               image  emotion
0  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...      sad
1  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...    anger
2  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...  neutral
3  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...     fear
4  {'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...  content


Now we just have images in the first column with the emotion in the second column.

In [32]:
from sklearn.model_selection import train_test_split

# Separate feature (X) and label (y)
X = df['image']
y = df['emotion']

# Perform a stratified split to keep class distribution consistent
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,      # 80% training, 20% testing
    random_state=42,    # for reproducibility
    stratify=y          # important for classification
)

# Validation split from X_train if needed:
X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.25,     # 25% of the training set (which is 20% of the total) -> 15% overall
    random_state=42,
    stratify=y_train
)

print("Training set size:", len(X_train))
print("Test set size:", len(X_test))
print("Validation set size:", len(X_val))

Training set size: 5640
Test set size: 1880
Validation set size: 1880


In [33]:
# Image bytes -> numpy arrays
def decode_images(image_series, target_size=(224, 224)):
    """
    Takes a pandas Series of dictionaries, each containing {'bytes': ...}.
    Decodes them into a list of NumPy arrays (RGB).
    Resizes images to target_size.
    Normalizes pixel values to [0, 1].

    Returns:
      - A NumPy array of shape (num_samples, target_size[0], target_size[1], 3)
    """
    decoded_list = []
    for item in image_series:
        # item should be a dict like {'bytes': b'...'}
        try:
            img_bytes = item['bytes']
            with Image.open(io.BytesIO(img_bytes)) as img:
                # Convert to RGB if needed
                img = img.convert('RGB')
                # Resize
                img = img.resize(target_size)
                # Convert to array
                arr = np.array(img, dtype=np.float32) / 255.0
            decoded_list.append(arr)
        except Exception as e:
            # If there's a bad image, you might want to handle or skip it
            print("Error decoding image:", e)
            # Optionally skip or handle it somehow. For now, let's skip:
            # Continue with the loop
            continue

    return np.stack(decoded_list, axis=0)

print("\nDecoding and resizing images...")

# Decode train set
X_train_array = decode_images(X_train, target_size=(224, 224))
print("X_train_array shape:", X_train_array.shape)

# Decode val set
X_val_array = decode_images(X_val, target_size=(224, 224))
print("X_val_array shape:", X_val_array.shape)

# Decode test set
X_test_array = decode_images(X_test, target_size=(224, 224))
print("X_test_array shape:", X_test_array.shape)


Decoding and resizing images...
X_train_array shape: (5640, 224, 224, 3)
X_val_array shape: (1880, 224, 224, 3)
X_test_array shape: (1880, 224, 224, 3)


In [34]:
# Encode labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded   = label_encoder.transform(y_val)
y_test_encoded  = label_encoder.transform(y_test)

print("\nLabel classes found:", label_encoder.classes_)
print("Sample of encoded labels:", y_train_encoded[:10])


Label classes found: ['anger' 'content' 'disgust' 'fear' 'happy' 'neutral' 'sad' 'surprise']
Sample of encoded labels: [6 3 5 6 0 6 7 3 4 0]


In [35]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Suppose you have: (224, 224, 3) images
# If you used a different size (e.g. 160x160 for MobileNet), be consistent

num_classes = len(label_encoder.classes_)

# 1) Load a MobileNetV2 (or EfficientNet, ResNet, etc.) without its top layers
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

# 2) Freeze the base_model so we only train the new head first
base_model.trainable = False

# Define a simple data augmentation pipeline:
data_augmentation = tf.keras.Sequential([
    # Random rotation: factor=0.25 corresponds roughly to ±90° rotation.
    layers.RandomRotation(0.25),
    # Random horizontal flip.
    layers.RandomFlip("horizontal")
])

# 3) Build your classifier on top
model = models.Sequential([
    tf.keras.Input(shape=(224,224,3)),
    # Data augmentation layers (only active during training).
    data_augmentation,
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

# 4) Train the new top layers
history = model.fit(
    X_train_array, y_train_encoded,
    validation_data=(X_val_array, y_val_encoded),
    epochs=5,
    batch_size=32
)

# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test_array, y_test_encoded)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

# 5) (Optional) Fine-tune deeper layers
# Unfreeze part (or all) of base_model and re-compile with a lower learning rate
base_model.trainable = True
# You can selectively unfreeze only some layers:
# for layer in base_model.layers[:100]:
#     layer.trainable = False

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(1e-5),  # smaller LR for fine-tuning
    metrics=['accuracy']
)

history_fine = model.fit(
    X_train_array, y_train_encoded,
    validation_data=(X_val_array, y_val_encoded),
    epochs=5,
    batch_size=32
)

test_loss, test_acc = model.evaluate(X_test_array, y_test_encoded)
print(f"\nFinal Test Loss after fine-tuning: {test_loss:.4f}")
print(f"Final Test Accuracy after fine-tuning: {test_acc:.4f}")

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_6 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │        10,248 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,268,232 (8.65 MB)

 Trainable params: 10,248 (40.03 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/5
177/177 ━━━━━━━━━━━━━━━━━━━━ 23s 117ms/step - accuracy: 0.1871 - loss: 2.1967 - val_accuracy: 0.2691 - val_loss: 1.8935
Epoch 2/5
177/177 ━━━━━━━━━━━━━━━━━━━━ 19s 105ms/step - accuracy: 0.2599 - loss: 1.9593 - val_accuracy: 0.3181 - val_loss: 1.8175
Epoch 3/5
177/177 ━━━━━━━━━━━━━━━━━━━━ 18s 100ms/step - accuracy: 0.2965 - loss: 1.8548 - val_accuracy: 0.3324 - val_loss: 1.7996
Epoch 4/5
177/177 ━━━━━━━━━━━━━━━━━━━━ 18s 101ms/step - accuracy: 0.3362 - loss: 1.7889 - val_accuracy: 0.3319 - val_loss: 1.7817
Epoch 5/5
177/177 ━━━━━━━━━━━━━━━━━━━━ 18s 102ms/step - accuracy: 0.3518 - loss: 1.7505 - val_accuracy: 0.3277 - val_loss: 1.7997
59/59 ━━━━━━━━━━━━━━━━━━━━ 4s 68ms/step - accuracy: 0.3215 - loss: 1.8659

Test Loss: 1.8364
Test Accuracy: 0.3303
Epoch 1/5
177/177 ━━━━━━━━━━━━━━━━━━━━ 111s 484ms/step - accuracy: 0.2656 - loss: 2.0305 - val_accuracy: 0.2963 - val_loss: 1.9705
Epoch 2/5
177/177 ━━━━━━━━━━━━━━━━━━━━ 82s 460ms/step - accuracy: 0.3104 - loss: 1.8336 - val_accuracy: 

In [36]:
# Save the model in the native TensorFlow SavedModel format.
model.save("saved_MobileNetV2.keras")

# Load the model later.
loaded_model = tf.keras.models.load_model("saved_MobileNetV2.keras")

In [39]:
model = tf.keras.models.load_model("saved_MobileNetV2.keras")

In [40]:
# [21] cell

import numpy as np
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt

# 1) Load the PREVIOUSLY trained model from disk:
model = tf.keras.models.load_model("saved_MobileNetV2.keras")

def get_gradcam_heatmap(model, image, last_conv_layer_name="block_16_project"):
    try:
        last_conv_layer = model.get_layer(last_conv_layer_name)
    except ValueError:
        base_model = model.get_layer("mobilenetv2_1.00_224")
        last_conv_layer = base_model.get_layer(last_conv_layer_name)

    grad_model = tf.keras.models.Model(
        inputs=model.input,
        outputs=[last_conv_layer.output, model.output]
    )
    image_batch = np.expand_dims(image, axis=0)
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(image_batch)
        pred_index = tf.argmax(predictions[0])
        loss = predictions[:, pred_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap)+1e-10)
    return heatmap.numpy()

def superimpose_heatmap(original_img, heatmap, alpha=0.4):
    if original_img.max() <= 1.0:
        original_img = (original_img * 255).astype("uint8")
    else:
        original_img = original_img.astype("uint8")
    heatmap_resized = cv2.resize(heatmap, 
                                 (original_img.shape[1], original_img.shape[0]))
    heatmap_resized = np.uint8(255 * heatmap_resized)
    jet = cv2.applyColorMap(heatmap_resized, cv2.COLORMAP_JET)
    return cv2.addWeighted(jet, alpha, original_img, 1 - alpha, 0)

# 2) IMPORTANT: call the model once:
dummy_input = np.zeros((1,224,224,3), dtype="float32")
_ = model(dummy_input)

# 3) Now do Grad-CAM:
test_image = np.random.rand(224,224,3).astype("float32")
heatmap = get_gradcam_heatmap(model, test_image, "block_16_project")
superimposed = superimpose_heatmap(test_image, heatmap)
plt.imshow(cv2.cvtColor(superimposed, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.show()

AttributeError: The layer sequential_7 has never been called and thus has no defined input.